In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
import requests
import time
from random import uniform
import pandas as pd

In [2]:
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)
#여성 상의
#driver.get('https://shopping.naver.com/style/style/category?menu=10000532')
# 여성 블라우스/셔츠
driver.get('https://shopping.naver.com/style/style/category?menu=10000533')
# 여성 하의 스커트
#driver.get('https://shopping.naver.com/style/style/category?menu=10000548')
#남성
#driver.get('https://shopping.naver.com/style/style/category?menu=10000623')
# 남성 셔츠/블라우스
#driver.get('https://shopping.naver.com/style/style/category?menu=10000631')

In [3]:
# 스크롤
link_list = []

for scr in range(10):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select('div._1aNmedx3I0 > ul._3KH4iCkMJ_ > li > div > a')
    f = uniform(2.0, 2.5)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    for link in links:
        link_list.append('https://shopping.naver.com'+link['href'])
        
    
    if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(f)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue

In [4]:
link_list

['https://shopping.naver.com/style/style/stores/100065193/products/4863854607',
 'https://shopping.naver.com/style/style/stores/1000010363/products/4863852364',
 'https://shopping.naver.com/style/style/stores/100147689/products/4863849556',
 'https://shopping.naver.com/style/style/stores/100286575/products/4863844424',
 'https://shopping.naver.com/style/style/stores/100431157/products/4863839090',
 'https://shopping.naver.com/style/style/stores/100114520/products/4843841089#INTRODUCE',
 'https://shopping.naver.com/style/style/stores/100135764/products/4863836265',
 'https://shopping.naver.com/style/style/stores/100528009/products/4862493850',
 'https://shopping.naver.com/style/style/stores/100065193/products/4863827630',
 'https://shopping.naver.com/style/style/stores/100153846/products/4863824455',
 'https://shopping.naver.com/style/style/stores/100118537/products/4863819744',
 'https://shopping.naver.com/style/style/stores/100179310/products/4856820273#INTRODUCE',
 'https://shopping.

In [5]:
link_list = list(set(link_list))

In [6]:
# csv 파일로 저장
import pandas as pd
results = pd.DataFrame(link_list, columns=['link_addr'])
results.to_csv('./link_addr_W_b$s.csv', header=False) #, mode='a', header=False)

In [7]:
#csv 파일 가져오기
import csv
f = open('./link_addr_W_b$s.csv','rt')
link_addr_file = csv.reader(f)

In [8]:
link_list = []
for row in link_addr_file :
    link_list.append(row[1])

In [9]:
#상세 페이지 들어가서 원하는 정보 저장
i = 0

while i < len(link_list):
    text_list = []
    
    driver.get(link_list[i])
    print(link_list[i])
    
    f = uniform(1.5, 1.7)
    time.sleep(f)
    
    a = ""
    a = link_list[i].split("/products/")
    a[1][:10]
    
    i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        images = soup.find('div','_396uGDX0Fi')
        # soup.select -> soup.find
        # ('div._3utq7rRVBL > div.editor_wrap > div > div > div.se_component_wrap > div.se_component > 
        # div.se_sectionArea > div.se_editArea > div.se_viewArea > a > img')
        name = soup.find('h3','_2IA5sp7BRM').get_text() # 동일
        price = soup.find('strong','WvptIJtvXo').get_text()  
        tag=soup.find('div','_24FKuaBdoK').get_text() # 동일
        info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]  # 동일
        detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)] # 동일
        texts = soup.find_all('p', 'se_textarea') #동일


        if not os.path.isdir(a[1][:10]):
            os.mkdir(a[1][:10])     

        for text in texts:
            text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
            text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = ({'category':'여성/상의/셔츠_남방', 'name': [name], 'price': [price] , 'tag':[tag], 'size':[text_list2]})   #, 'price': [price]
        df1 = pd.DataFrame(data_1)

        data_2 = [detail]
        df2 = pd.DataFrame(data_2, columns = info)

        df3 = pd.concat([df1,df2], axis=1)

        df3.to_csv(".\\"+a[1][:10]+"\\csv_"+a[1][:10]+".csv", index = True, encoding='utf-8')

        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://www.naver.com/"})
            src_image = src_page.content
            with open(".\\"+a[1][:10]+"\\img_"+a[1][:10]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)

    except:
        print("상품정보 가져오기 오류발생")
        pass

https://shopping.naver.com/style/style/stores/100262851/products/4863396035
https://shopping.naver.com/style/style/stores/100013780/products/4863317224
https://shopping.naver.com/style/style/stores/100065193/products/4863854607#INTRODUCE
https://shopping.naver.com/style/style/stores/1000019900/products/4862358259
https://shopping.naver.com/style/style/stores/100093473/products/4863460179
https://shopping.naver.com/style/style/stores/100160893/products/4863654858
https://shopping.naver.com/style/style/stores/100193113/products/4863364280
https://shopping.naver.com/style/style/stores/100114520/products/4848098342#INTRODUCE
https://shopping.naver.com/style/style/stores/100206575/products/4863357196
https://shopping.naver.com/style/style/stores/100013780/products/4863318325
https://shopping.naver.com/style/style/stores/100348738/products/4863323465
https://shopping.naver.com/style/style/stores/1000010153/products/4863580076
https://shopping.naver.com/style/style/stores/100067106/products/4

https://shopping.naver.com/style/style/stores/100675779/products/4863077474
https://shopping.naver.com/style/style/stores/100655395/products/4863029926
https://shopping.naver.com/style/style/stores/1000021456/products/4862214225
https://shopping.naver.com/style/style/stores/100013780/products/4863533854
https://shopping.naver.com/style/style/stores/100670135/products/4860361661
https://shopping.naver.com/style/style/stores/100159029/products/4863430307
https://shopping.naver.com/style/style/stores/100646459/products/4861594890
https://shopping.naver.com/style/style/stores/1000015448/products/4863732704
https://shopping.naver.com/style/style/stores/1000021186/products/4863356102
https://shopping.naver.com/style/style/stores/100013780/products/4863533802
https://shopping.naver.com/style/style/stores/1000002598/products/4863360335
https://shopping.naver.com/style/style/stores/100118537/products/4863557553
https://shopping.naver.com/style/style/stores/100041826/products/4863615134
https://

https://shopping.naver.com/style/style/stores/1000009794/products/4861624761#INTRODUCE
https://shopping.naver.com/style/style/stores/1000019900/products/4861501822
https://shopping.naver.com/style/style/stores/1000004872/products/4863503773
https://shopping.naver.com/style/style/stores/1000021459/products/4863613346
https://shopping.naver.com/style/style/stores/100013780/products/4863759563
https://shopping.naver.com/style/style/stores/100085201/products/4863540614
https://shopping.naver.com/style/style/stores/1000003779/products/4863306223
https://shopping.naver.com/style/style/stores/1000017157/products/4863485274
https://shopping.naver.com/style/style/stores/100206046/products/4863605083
